In [13]:
import numpy as np
import pandas as pd
import data_lib
import plot_lib
import decision_lib
import validation_lib
import transform_lib
from icecream import ic
import stats_lib 

np.random.seed(200)

In [14]:
# print available data summary
data_dict = data_lib.explore_datasets(datafolder="../../Data",verbose=True)
print(data_lib.LABELS_LIST)

----------------------------------------------------------------------------------------------
-- The following 4 groups were found
-- They contain 40 datasets
-- The first printed entity is the key to the returned dictionary
-----------------------------------
Group: ../../Data/6P-positive-dilution-series-2-labelled/droplet-level-data/RawData
po-di-se-2-A4, files: 13                po-di-se-2-C4, files: 13                po-di-se-2-A1, files: 13
po-di-se-2-B1, files: 13                po-di-se-2-D1, files: 13                po-di-se-2-B4, files: 13
po-di-se-2-C1, files: 13                po-di-se-2-D4, files: 13                
-----------------------------------
Group: ../../Data/6P-positive-dilution-series-1-labelled/droplet-level-data/RawData
po-di-se-1-D4, files: 13                po-di-se-1-A4, files: 13                po-di-se-1-A1, files: 13
po-di-se-1-D1, files: 13                po-di-se-1-B1, files: 13                po-di-se-1-C1, files: 13
po-di-se-1-B4, files: 13         

### Get samples for negative control

In [15]:
# get all discovered datasets
file_list = data_dict.keys()

# mark negative contorls
negative_control_list =[
                        "wa-sa-D3",
                        "wa-sa-D5",
                        "po-di-se-1-D1",
                        "po-di-se-1-D4",
                        "po-di-se-2-D1", 
                        "po-di-se-2-D4",
                        "po-di-se-D8",
                       ] 


In [16]:
# fix clustering algorithm
prediction_axis = ['SARS-N2_POS','SARS-N1_POS','IBV-M_POS','RSV-N_POS','IAV-M_POS','MHV_POS']
neg_list = []
pos_list = []
neg_control_list = []
for file in file_list:
    df = data_lib.load_dataset(prediction_axis,  [file], datafolder="../../Data")
    np_data = df.to_numpy()
    np_chan = np_data[:,:6]
    np_labels = np_data[:,6:]
    n = np_data.shape[0]
    active = ~(np.sum(np_labels, axis=0) / n == 0)
    certainty, _, statistic = decision_lib.get_negative_dimensions(np_chan, 0.001)
    statistic = statistic / n
    exceptions = {
        "wa-sa-B3" : np.array([True, True, True, False, False, True], dtype=bool),
        "wa-sa-A2" : np.array([True, True, True, False, False, True], dtype=bool),
        "wa-sa-B2" : np.array([True, True, True, False, False, True], dtype=bool),
        "wa-sa-A3" : np.array([True, True, True, False, False, True], dtype=bool),
        "wa-sa-B1" : np.array([True, True, True, False, False, True], dtype=bool),
        "wa-sa-A1" : np.array([True, True, True, False, False, True], dtype=bool),

        "wa-sa-C2" : np.array([True, True, True, False, False, False], dtype=bool),
        "wa-sa-D1" : np.array([True, True, True, False, False, False], dtype=bool),
        "wa-sa-D2" : np.array([True, True, True, False, False, False], dtype=bool),
        "wa-sa-C1" : np.array([True, True, True, False, False, False], dtype=bool),

        "wa-sa-B4" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-C4" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-D3" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-D4" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-D5" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-A4" : np.array([False, False,False, False, False, False], dtype=bool),
        "wa-sa-B5" : np.array([False, False,False, False, False, False], dtype=bool),
        
        "po-di-se-1-D1" : np.array([False, False,False, False, False, False], dtype=bool),
        "po-di-se-D8" : np.array([False, False,False, False, False, False], dtype=bool),
                  }
    if file in exceptions.keys():
        active = exceptions[file]
    if file in negative_control_list:
        neg_control_list.append(statistic)
    neg_list.append(statistic[~active])
    pos_list.append(statistic[active])
    if np.any(active) and np.any(~active) and (not file in exceptions.keys()):
        ic(file, certainty, statistic, active)

    #df_data_points = pd.DataFrame(data=decision.X_transformed, columns=prediction_axis) 
ic(np.mean(np.concatenate(pos_list)))
ic(np.std(np.concatenate(pos_list)))
ic(np.mean(np.concatenate(neg_list)))
ic(np.std(np.concatenate(neg_list)))
ic(np.mean(np.concatenate(neg_control_list)))
ic(np.std(np.concatenate(neg_control_list)))

ic| np.mean(np.concatenate(pos_list)): 1.546217853889672
ic| np.std(np.concatenate(pos_list)): 1.2634362717927208
ic| np.mean(np.concatenate(neg_list)): 0.07794832879658467
ic| np.std(np.concatenate(neg_list)): 0.08847322513795011
ic| np.mean(np.concatenate(neg_control_list)): 0.03647890062131418
ic| np.std(np.concatenate(neg_control_list)): 0.034002712919506496


0.034002712919506496